In [1]:
import os
import sys

from dotenv import load_dotenv

sys.path.append(os.path.abspath(".."))

load_dotenv("/Users/steffen/agentic_ai/.env")

True

In [5]:
from src.agent.adapters.sql_tools.controller import create_sql_controller

controller = create_sql_controller()

In [6]:
questions = [
    "How many customers do we have?",
    "What are the top 5 selling products?",
    "Show me orders from 2024",
    "What is the average order value?",
]

In [7]:
i = 3

In [8]:
result = controller.process_question(questions[i])

2025-07-05 16:01:52.129 | INFO     | src.agent.adapters.database:_get_connection:77 - SQLAlchemy engine created successfully.
2025-07-05 16:01:52.130 | INFO     | src.agent.adapters.database:connect:90 - Connected to postgres database
2025-07-05 16:01:52.350 | INFO     | src.agent.adapters.database:disconnect:101 - Disconnected from database
Authentication error: Langfuse client initialized without public_key. Client will be disabled. Provide a public_key parameter or set LANGFUSE_PUBLIC_KEY environment variable. 
Authentication error: Langfuse client initialized without public_key. Client will be disabled. Provide a public_key parameter or set LANGFUSE_PUBLIC_KEY environment variable. 
Context error: No active span in current context. Operations that depend on an active span will be skipped. Ensure spans are created with start_as_current_span() or that you're operating within an active span context.
Authentication error: Langfuse client initialized without public_key. Client will be d

In [9]:
result.confidence

1.0

In [7]:
print(result.sql_query)

AttributeError: 'coroutine' object has no attribute 'sql_query'

## Evaluate the SQL agent

#### Easy

In [8]:
pairs = [
    (
        "What are the names and symbols of all available currencies?",
        """SELECT currency_code, name, symbol
FROM currencies;
""",
    ),
    (
        "List all employees along with their role name and the territory they work in.",
        """SELECT e.name AS employee_name, r.name AS role_name, t.name AS territory_name
FROM employees e
JOIN roles r ON e.role_id = r.role_id
JOIN territories t ON e.territory_id = t.territory_id;
""",
    ),
    (
        "What are the current prices (latest by effective_date) of each product in USD?",
        """SELECT p.name AS product_name, pp.price, pp.effective_date
FROM products p
JOIN product_prices pp ON p.product_id = pp.product_id
WHERE pp.currency_code = 'USD'
  AND pp.end_date IS NULL;
""",
    ),
    (
        """Find the name and email of all employees who have the role of 'Sales Representative""",
        """SELECT
    e.name,
    e.email,
    r.name AS role_name
FROM
    employees e
JOIN
    roles r ON e.role_id = r.role_id
WHERE
    r.name = 'Sales Representative';""",
    ),
    (
        """List all products that are below their reorder level in any territory""",
        """SELECT
    p.name AS product_name,
    p.sku,
    t.name AS territory_name,
    i.quantity_on_hand,
    i.reorder_level
FROM
    inventory i
JOIN
    products p ON i.product_id = p.product_id
JOIN
    territories t ON i.territory_id = t.territory_id
WHERE
    i.quantity_on_hand < i.reorder_level;""",
    ),
    (
        """What are the 5 most recent orders? Show the order number, customer's company name, order date, and the grand total""",
        """SELECT
    o.order_number,
    c.company_name,
    o.order_date,
    o.grand_total_amount,
    o.currency_code
FROM
    orders o
JOIN
    customers c ON o.customer_id = c.customer_id
ORDER BY
    o.order_date DESC
LIMIT 5;""",
    ),
    (
        """What are all the active products in our catalog""",
        """SELECT
    product_id,
    name,
    sku
FROM products
WHERE is_active = TRUE
ORDER BY name;""",
    ),
    (
        """Which employees work in each territory?""",
        """SELECT
    t.name AS territory_name,
    e.name AS employee_name,
    e.email,
    r.name AS role
FROM employees e
JOIN territories t ON e.territory_id = t.territory_id
JOIN roles r ON e.role_id = r.role_id
WHERE e.is_active = TRUE
ORDER BY t.name, e.name;""",
    ),
    (
        """What is the current inventory level for each product?""",
        """SELECT
    p.name AS product_name,
    p.sku,
    i.quantity_on_hand,
    i.quantity_on_order,
    i.quantity_reserved,
    t.name AS territory_name
FROM inventory i
JOIN products p ON i.product_id = p.product_id
JOIN territories t ON i.territory_id = t.territory_id
ORDER BY p.name, t.name;""",
    ),
]

In [9]:
result = controller.process_question(pairs[0])

In [10]:
print(pairs[0][0])
print("-" * 100)

print(result.sql_query)
print("-" * 100)
print(pairs[0][1])

What are the names and symbols of all available currencies?
----------------------------------------------------------------------------------------------------


AttributeError: 'coroutine' object has no attribute 'sql_query'

In [47]:
i = 1
result = controller.process_question(pairs[i])

2025-07-04 10:37:23.784 | INFO     | src.agent.adapters.database:_get_connection:77 - SQLAlchemy engine created successfully.
2025-07-04 10:37:23.784 | INFO     | src.agent.adapters.database:connect:90 - Connected to postgres database
2025-07-04 10:37:23.850 | INFO     | src.agent.adapters.database:disconnect:101 - Disconnected from database
Authentication error: Langfuse client initialized without public_key. Client will be disabled. Provide a public_key parameter or set LANGFUSE_PUBLIC_KEY environment variable. 
Authentication error: Langfuse client initialized without public_key. Client will be disabled. Provide a public_key parameter or set LANGFUSE_PUBLIC_KEY environment variable. 
Context error: No active span in current context. Operations that depend on an active span will be skipped. Ensure spans are created with start_as_current_span() or that you're operating within an active span context.
Authentication error: Langfuse client initialized without public_key. Client will be d

In [49]:
print(pairs[i][0])
print("-" * 100)

print(result.sql_query)
print("-" * 100)
print(pairs[i][1])

List all employees along with their role name and the territory they work in.
----------------------------------------------------------------------------------------------------
SELECT e.name AS employee_name, r.name AS role_name, t.name AS territory_name
FROM employees e
JOIN roles r ON e.role_id = r.role_id
JOIN territories t ON e.territory_id = t.territory_id;
----------------------------------------------------------------------------------------------------
SELECT e.name AS employee_name, r.name AS role_name, t.name AS territory_name
FROM employees e
JOIN roles r ON e.role_id = r.role_id
JOIN territories t ON e.territory_id = t.territory_id;



In [50]:
i = 2
result = controller.process_question(pairs[i])

2025-07-04 10:38:55.437 | INFO     | src.agent.adapters.database:_get_connection:77 - SQLAlchemy engine created successfully.
2025-07-04 10:38:55.437 | INFO     | src.agent.adapters.database:connect:90 - Connected to postgres database
2025-07-04 10:38:55.569 | INFO     | src.agent.adapters.database:disconnect:101 - Disconnected from database
Authentication error: Langfuse client initialized without public_key. Client will be disabled. Provide a public_key parameter or set LANGFUSE_PUBLIC_KEY environment variable. 
Authentication error: Langfuse client initialized without public_key. Client will be disabled. Provide a public_key parameter or set LANGFUSE_PUBLIC_KEY environment variable. 
Context error: No active span in current context. Operations that depend on an active span will be skipped. Ensure spans are created with start_as_current_span() or that you're operating within an active span context.
Authentication error: Langfuse client initialized without public_key. Client will be d

In [51]:
print(pairs[i][0])
print("-" * 100)

print(result.sql_query)
print("-" * 100)
print(pairs[i][1])

What are the current prices (latest by effective_date) of each product in USD?
----------------------------------------------------------------------------------------------------
SELECT 
    p.name AS product_name,
    pp.price,
    pp.effective_date
FROM products p
JOIN (
    SELECT 
        product_id,
        price,
        effective_date,
        ROW_NUMBER() OVER (PARTITION BY product_id ORDER BY effective_date DESC) as rn
    FROM product_prices
    WHERE currency_code = 'USD'
      AND end_date IS NULL
) pp ON p.product_id = pp.product_id
WHERE pp.rn = 1
ORDER BY p.name;
----------------------------------------------------------------------------------------------------
SELECT p.name AS product_name, pp.price, pp.effective_date
FROM products p
JOIN product_prices pp ON p.product_id = pp.product_id
WHERE pp.currency_code = 'USD'
  AND pp.end_date IS NULL;



In [52]:
i = 4
result = controller.process_question(pairs[i])

2025-07-04 10:42:21.166 | INFO     | src.agent.adapters.database:_get_connection:77 - SQLAlchemy engine created successfully.
2025-07-04 10:42:21.167 | INFO     | src.agent.adapters.database:connect:90 - Connected to postgres database
2025-07-04 10:42:21.304 | INFO     | src.agent.adapters.database:disconnect:101 - Disconnected from database
Authentication error: Langfuse client initialized without public_key. Client will be disabled. Provide a public_key parameter or set LANGFUSE_PUBLIC_KEY environment variable. 
Authentication error: Langfuse client initialized without public_key. Client will be disabled. Provide a public_key parameter or set LANGFUSE_PUBLIC_KEY environment variable. 
Context error: No active span in current context. Operations that depend on an active span will be skipped. Ensure spans are created with start_as_current_span() or that you're operating within an active span context.
Authentication error: Langfuse client initialized without public_key. Client will be d

In [53]:
print(pairs[i][0])
print("-" * 100)

print(result.sql_query)
print("-" * 100)
print(pairs[i][1])

List all products that are below their reorder level in any territory
----------------------------------------------------------------------------------------------------
SELECT
    p.name AS product_name,
    p.sku,
    t.name AS territory_name,
    i.quantity_on_hand,
    i.reorder_level
FROM
    inventory i
JOIN
    products p ON i.product_id = p.product_id
JOIN
    territories t ON i.territory_id = t.territory_id
WHERE
    i.quantity_on_hand < i.reorder_level;
----------------------------------------------------------------------------------------------------
SELECT
    p.name AS product_name,
    p.sku,
    t.name AS territory_name,
    i.quantity_on_hand,
    i.reorder_level
FROM
    inventory i
JOIN
    products p ON i.product_id = p.product_id
JOIN
    territories t ON i.territory_id = t.territory_id
WHERE
    i.quantity_on_hand < i.reorder_level;


In [54]:
i = 6
result = controller.process_question(pairs[i])

2025-07-04 10:43:36.661 | INFO     | src.agent.adapters.database:_get_connection:77 - SQLAlchemy engine created successfully.
2025-07-04 10:43:36.662 | INFO     | src.agent.adapters.database:connect:90 - Connected to postgres database
2025-07-04 10:43:36.735 | INFO     | src.agent.adapters.database:disconnect:101 - Disconnected from database
Authentication error: Langfuse client initialized without public_key. Client will be disabled. Provide a public_key parameter or set LANGFUSE_PUBLIC_KEY environment variable. 
Authentication error: Langfuse client initialized without public_key. Client will be disabled. Provide a public_key parameter or set LANGFUSE_PUBLIC_KEY environment variable. 
Context error: No active span in current context. Operations that depend on an active span will be skipped. Ensure spans are created with start_as_current_span() or that you're operating within an active span context.
Authentication error: Langfuse client initialized without public_key. Client will be d

In [55]:
print(pairs[i][0])
print("-" * 100)

print(result.sql_query)
print("-" * 100)
print(pairs[i][1])

What are all the active products in our catalog
----------------------------------------------------------------------------------------------------
SELECT
    product_id,
    name,
    sku
FROM products
WHERE is_active = TRUE
ORDER BY name;
----------------------------------------------------------------------------------------------------
SELECT
    product_id,
    name,
    sku
FROM products
WHERE is_active = TRUE
ORDER BY name;


In [56]:
i = 8
result = controller.process_question(pairs[i])

2025-07-04 10:44:21.643 | INFO     | src.agent.adapters.database:_get_connection:77 - SQLAlchemy engine created successfully.
2025-07-04 10:44:21.644 | INFO     | src.agent.adapters.database:connect:90 - Connected to postgres database
2025-07-04 10:44:21.717 | INFO     | src.agent.adapters.database:disconnect:101 - Disconnected from database
Authentication error: Langfuse client initialized without public_key. Client will be disabled. Provide a public_key parameter or set LANGFUSE_PUBLIC_KEY environment variable. 
Authentication error: Langfuse client initialized without public_key. Client will be disabled. Provide a public_key parameter or set LANGFUSE_PUBLIC_KEY environment variable. 
Context error: No active span in current context. Operations that depend on an active span will be skipped. Ensure spans are created with start_as_current_span() or that you're operating within an active span context.
Authentication error: Langfuse client initialized without public_key. Client will be d

In [57]:
print(pairs[i][0])
print("-" * 100)

print(result.sql_query)
print("-" * 100)
print(pairs[i][1])

What is the current inventory level for each product?
----------------------------------------------------------------------------------------------------
SELECT
    p.name AS product_name,
    p.sku,
    i.quantity_on_hand,
    i.quantity_on_order,
    i.quantity_reserved,
    t.name AS territory_name
FROM inventory i
JOIN products p ON i.product_id = p.product_id
JOIN territories t ON i.territory_id = t.territory_id
ORDER BY p.name, t.name;
----------------------------------------------------------------------------------------------------
SELECT
    p.name AS product_name,
    p.sku,
    i.quantity_on_hand,
    i.quantity_on_order,
    i.quantity_reserved,
    t.name AS territory_name
FROM inventory i
JOIN products p ON i.product_id = p.product_id
JOIN territories t ON i.territory_id = t.territory_id
ORDER BY p.name, t.name;


#### Medium

In [60]:
pairs = [
    (
        "What are the current tax rates per country and territory, including tax type names?",
        """SELECT
    co.name AS country_name,
    te.name AS territory_name,
    tt.name AS tax_type,
    tr.rate,
    tr.effective_date
FROM tax_rates tr
JOIN tax_types tt ON tr.tax_type_id = tt.tax_type_id
LEFT JOIN countries co ON tr.country_id = co.country_id
LEFT JOIN territories te ON tr.territory_id = te.territory_id
WHERE tr.end_date IS NULL;
""",
    ),
    (
        "Find all employees who report to a manager, showing their name and their manager's name.",
        """SELECT
    e.name AS employee_name,
    m.name AS manager_name
FROM employees e
JOIN employees m ON e.manager_id = m.employee_id;
""",
    ),
    (
        "Show each product's latest cost (by effective_date) in its original currency, including cost type.",
        """SELECT
    p.name AS product_name,
    ct.name AS cost_type,
    pc.amount,
    pc.currency_code,
    pc.effective_date
FROM product_costs pc
JOIN (
    SELECT product_id, cost_type_id, MAX(effective_date) AS latest_date
    FROM product_costs
    WHERE end_date IS NULL
    GROUP BY product_id, cost_type_id
) latest ON pc.product_id = latest.product_id
         AND pc.cost_type_id = latest.cost_type_id
         AND pc.effective_date = latest.latest_date
JOIN products p ON pc.product_id = p.product_id
JOIN cost_types ct ON pc.cost_type_id = ct.cost_type_id;
""",
    ),
    (
        """Calculate the total revenue generated by each 'Sales Representative' in the year 2023. List the employee's name and their total sales amount, ordered from highest to lowest""",
        """SELECT
    e.name AS employee_name,
    SUM(o.grand_total_amount) AS total_revenue
FROM
    orders o
JOIN
    employees e ON o.employee_id = e.employee_id
JOIN
    roles r ON e.role_id = r.role_id
WHERE
    r.name = 'Sales Representative'
    AND EXTRACT(YEAR FROM o.order_date) = 2023
GROUP BY
    e.employee_id, e.name
ORDER BY
    total_revenue DESC;""",
    ),
    (
        """Find all employees who have a salary greater than their direct manager's salary. Show the employee's name and salary, alongside their manager's name and salary""",
        """SELECT
    e.name AS employee_name,
    SUM(o.grand_total_amount) AS total_revenue
FROM
    orders o
JOIN
    employees e ON o.employee_id = e.employee_id
JOIN
    roles r ON e.role_id = r.role_id
WHERE
    r.name = 'Sales Representative'
    AND EXTRACT(YEAR FROM o.order_date) = 2023
GROUP BY
    e.employee_id, e.name
ORDER BY
    total_revenue DESC;""",
    ),
    (
        """List the names and SKUs of all products that are supplied by more than one active supplier""",
        """SELECT
    p.name,
    p.sku
FROM
    products p
WHERE
    p.product_id IN (
        SELECT
            ps.product_id
        FROM
            product_suppliers ps
        JOIN
            suppliers s ON ps.supplier_id = s.supplier_id
        WHERE
            s.is_active = TRUE
        GROUP BY
            ps.product_id
        HAVING
            COUNT(ps.supplier_id) > 1
    );""",
    ),
    (
        """What is the total value of all received purchase orders by supplier?""",
        """SELECT
    s.company_name AS supplier_name,
    COUNT(DISTINCT po.po_id) AS number_of_orders,
    SUM(pod.quantity * pod.unit_cost) AS total_value,
    po.currency_code
FROM purchase_orders po
JOIN suppliers s ON po.supplier_id = s.supplier_id
JOIN purchase_order_details pod ON po.po_id = pod.po_id
WHERE po.status = 'RECEIVED'
GROUP BY s.supplier_id, s.company_name, po.currency_code
ORDER BY total_value DESC;""",
    ),
    (
        """Which products have low inventory (below reorder level) and who are their preferred suppliers?""",
        """SELECT
    p.name AS product_name,
    p.sku,
    t.name AS territory_name,
    i.quantity_on_hand,
    i.reorder_level,
    s.company_name AS preferred_supplier,
    ps.unit_cost,
    ps.cost_currency_code,
    ps.lead_time_days
FROM inventory i
JOIN products p ON i.product_id = p.product_id
JOIN territories t ON i.territory_id = t.territory_id
LEFT JOIN product_suppliers ps ON p.product_id = ps.product_id
    AND ps.is_preferred = TRUE
    AND ps.end_date IS NULL
LEFT JOIN suppliers s ON ps.supplier_id = s.supplier_id
WHERE i.quantity_on_hand < i.reorder_level
ORDER BY t.name, p.name;""",
    ),
    (
        """What are the top 5 customers by order value im ersten halbjahr 2024, including their order count?""",
        """SELECT
    c.customer_id,
    c.company_name,
    COUNT(DISTINCT o.order_id) AS order_count,
    SUM(o.grand_total_amount) AS total_order_value,
    o.currency_code,
    AVG(o.grand_total_amount) AS average_order_value
FROM customers c
JOIN orders o ON c.customer_id = o.customer_id
WHERE o.order_date BETWEEN '2024-01-01' AND '2024-06-30'
    AND o.status NOT IN ('CANCELLED', 'PENDING')
GROUP BY c.customer_id, c.company_name, o.currency_code
ORDER BY total_order_value DESC
LIMIT 5;""",
    ),
]

In [61]:
result = controller.process_question(pairs[0])

2025-07-04 10:47:03.814 | INFO     | src.agent.adapters.database:_get_connection:77 - SQLAlchemy engine created successfully.
2025-07-04 10:47:03.815 | INFO     | src.agent.adapters.database:connect:90 - Connected to postgres database
2025-07-04 10:47:03.882 | INFO     | src.agent.adapters.database:disconnect:101 - Disconnected from database
Authentication error: Langfuse client initialized without public_key. Client will be disabled. Provide a public_key parameter or set LANGFUSE_PUBLIC_KEY environment variable. 
Authentication error: Langfuse client initialized without public_key. Client will be disabled. Provide a public_key parameter or set LANGFUSE_PUBLIC_KEY environment variable. 
Context error: No active span in current context. Operations that depend on an active span will be skipped. Ensure spans are created with start_as_current_span() or that you're operating within an active span context.
Authentication error: Langfuse client initialized without public_key. Client will be d

In [62]:
print(pairs[0][0])
print("-" * 100)

print(result.sql_query)
print("-" * 100)
print(pairs[0][1])

What are the current tax rates per country and territory, including tax type names?
----------------------------------------------------------------------------------------------------
SELECT
    co.name AS country_name,
    te.name AS territory_name,
    tt.name AS tax_type,
    tr.rate,
    tr.effective_date
FROM tax_rates tr
JOIN tax_types tt ON tr.tax_type_id = tt.tax_type_id
LEFT JOIN countries co ON tr.country_id = co.country_id
LEFT JOIN territories te ON tr.territory_id = te.territory_id
WHERE tr.end_date IS NULL;
----------------------------------------------------------------------------------------------------
SELECT
    co.name AS country_name,
    te.name AS territory_name,
    tt.name AS tax_type,
    tr.rate,
    tr.effective_date
FROM tax_rates tr
JOIN tax_types tt ON tr.tax_type_id = tt.tax_type_id
LEFT JOIN countries co ON tr.country_id = co.country_id
LEFT JOIN territories te ON tr.territory_id = te.territory_id
WHERE tr.end_date IS NULL;



In [63]:
i = 1
result = controller.process_question(pairs[i])

2025-07-04 10:48:15.936 | INFO     | src.agent.adapters.database:_get_connection:77 - SQLAlchemy engine created successfully.
2025-07-04 10:48:15.937 | INFO     | src.agent.adapters.database:connect:90 - Connected to postgres database
2025-07-04 10:48:16.066 | INFO     | src.agent.adapters.database:disconnect:101 - Disconnected from database
Authentication error: Langfuse client initialized without public_key. Client will be disabled. Provide a public_key parameter or set LANGFUSE_PUBLIC_KEY environment variable. 
Authentication error: Langfuse client initialized without public_key. Client will be disabled. Provide a public_key parameter or set LANGFUSE_PUBLIC_KEY environment variable. 
Context error: No active span in current context. Operations that depend on an active span will be skipped. Ensure spans are created with start_as_current_span() or that you're operating within an active span context.
Authentication error: Langfuse client initialized without public_key. Client will be d

In [64]:
print(pairs[i][0])
print("-" * 100)

print(result.sql_query)
print("-" * 100)
print(pairs[i][1])

Find all employees who report to a manager, showing their name and their manager's name.
----------------------------------------------------------------------------------------------------
SELECT
    e.name AS employee_name,
    m.name AS manager_name
FROM employees e
JOIN employees m ON e.manager_id = m.employee_id;
----------------------------------------------------------------------------------------------------
SELECT
    e.name AS employee_name,
    m.name AS manager_name
FROM employees e
JOIN employees m ON e.manager_id = m.employee_id;



In [ ]:
i = 2
result = controller.process_question(pairs[i])

2025-07-04 10:38:55.437 | INFO     | src.agent.adapters.database:_get_connection:77 - SQLAlchemy engine created successfully.
2025-07-04 10:38:55.437 | INFO     | src.agent.adapters.database:connect:90 - Connected to postgres database
2025-07-04 10:38:55.569 | INFO     | src.agent.adapters.database:disconnect:101 - Disconnected from database
Authentication error: Langfuse client initialized without public_key. Client will be disabled. Provide a public_key parameter or set LANGFUSE_PUBLIC_KEY environment variable. 
Authentication error: Langfuse client initialized without public_key. Client will be disabled. Provide a public_key parameter or set LANGFUSE_PUBLIC_KEY environment variable. 
Context error: No active span in current context. Operations that depend on an active span will be skipped. Ensure spans are created with start_as_current_span() or that you're operating within an active span context.
Authentication error: Langfuse client initialized without public_key. Client will be d

In [ ]:
print(pairs[i][0])
print("-" * 100)

print(result.sql_query)
print("-" * 100)
print(pairs[i][1])

What are the current prices (latest by effective_date) of each product in USD?
----------------------------------------------------------------------------------------------------
SELECT 
    p.name AS product_name,
    pp.price,
    pp.effective_date
FROM products p
JOIN (
    SELECT 
        product_id,
        price,
        effective_date,
        ROW_NUMBER() OVER (PARTITION BY product_id ORDER BY effective_date DESC) as rn
    FROM product_prices
    WHERE currency_code = 'USD'
      AND end_date IS NULL
) pp ON p.product_id = pp.product_id
WHERE pp.rn = 1
ORDER BY p.name;
----------------------------------------------------------------------------------------------------
SELECT p.name AS product_name, pp.price, pp.effective_date
FROM products p
JOIN product_prices pp ON p.product_id = pp.product_id
WHERE pp.currency_code = 'USD'
  AND pp.end_date IS NULL;



In [65]:
i = 4
result = controller.process_question(pairs[i])

2025-07-04 10:49:56.335 | INFO     | src.agent.adapters.database:_get_connection:77 - SQLAlchemy engine created successfully.
2025-07-04 10:49:56.335 | INFO     | src.agent.adapters.database:connect:90 - Connected to postgres database
2025-07-04 10:49:56.474 | INFO     | src.agent.adapters.database:disconnect:101 - Disconnected from database
Authentication error: Langfuse client initialized without public_key. Client will be disabled. Provide a public_key parameter or set LANGFUSE_PUBLIC_KEY environment variable. 
Authentication error: Langfuse client initialized without public_key. Client will be disabled. Provide a public_key parameter or set LANGFUSE_PUBLIC_KEY environment variable. 
Context error: No active span in current context. Operations that depend on an active span will be skipped. Ensure spans are created with start_as_current_span() or that you're operating within an active span context.
Authentication error: Langfuse client initialized without public_key. Client will be d

In [66]:
print(pairs[i][0])
print("-" * 100)

print(result.sql_query)
print("-" * 100)
print(pairs[i][1])

Find all employees who have a salary greater than their direct manager's salary. Show the employee's name and salary, alongside their manager's name and salary
----------------------------------------------------------------------------------------------------
SELECT 
    e.name AS employee_name,
    e.salary AS employee_salary,
    m.name AS manager_name,
    m.salary AS manager_salary
FROM 
    employees e
INNER JOIN 
    employees m ON e.manager_id = m.employee_id
WHERE 
    e.salary > m.salary
ORDER BY 
    e.salary DESC;
----------------------------------------------------------------------------------------------------
SELECT
    e.name AS employee_name,
    SUM(o.grand_total_amount) AS total_revenue
FROM
    orders o
JOIN
    employees e ON o.employee_id = e.employee_id
JOIN
    roles r ON e.role_id = r.role_id
WHERE
    r.name = 'Sales Representative'
    AND EXTRACT(YEAR FROM o.order_date) = 2023
GROUP BY
    e.employee_id, e.name
ORDER BY
    total_revenue DESC;


In [67]:
i = 6
result = controller.process_question(pairs[i])

2025-07-04 10:53:16.041 | INFO     | src.agent.adapters.database:_get_connection:77 - SQLAlchemy engine created successfully.
2025-07-04 10:53:16.042 | INFO     | src.agent.adapters.database:connect:90 - Connected to postgres database
2025-07-04 10:53:16.172 | INFO     | src.agent.adapters.database:disconnect:101 - Disconnected from database
Authentication error: Langfuse client initialized without public_key. Client will be disabled. Provide a public_key parameter or set LANGFUSE_PUBLIC_KEY environment variable. 
Authentication error: Langfuse client initialized without public_key. Client will be disabled. Provide a public_key parameter or set LANGFUSE_PUBLIC_KEY environment variable. 
Context error: No active span in current context. Operations that depend on an active span will be skipped. Ensure spans are created with start_as_current_span() or that you're operating within an active span context.
Authentication error: Langfuse client initialized without public_key. Client will be d

In [68]:
print(pairs[i][0])
print("-" * 100)

print(result.sql_query)
print("-" * 100)
print(pairs[i][1])

What is the total value of all received purchase orders by supplier?
----------------------------------------------------------------------------------------------------
SELECT
    s.company_name AS supplier_name,
    COUNT(DISTINCT po.po_id) AS number_of_orders,
    SUM(pod.quantity * pod.unit_cost) AS total_value
FROM purchase_orders po
JOIN suppliers s ON po.supplier_id = s.supplier_id
JOIN purchase_order_details pod ON po.po_id = pod.po_id
WHERE po.status = 'RECEIVED'
GROUP BY s.supplier_id, s.company_name
ORDER BY total_value DESC;
----------------------------------------------------------------------------------------------------
SELECT
    s.company_name AS supplier_name,
    COUNT(DISTINCT po.po_id) AS number_of_orders,
    SUM(pod.quantity * pod.unit_cost) AS total_value,
    po.currency_code
FROM purchase_orders po
JOIN suppliers s ON po.supplier_id = s.supplier_id
JOIN purchase_order_details pod ON po.po_id = pod.po_id
WHERE po.status = 'RECEIVED'
GROUP BY s.supplier_id, s.c

In [69]:
i = 8
result = controller.process_question(pairs[i])

2025-07-04 11:02:06.232 | INFO     | src.agent.adapters.database:_get_connection:77 - SQLAlchemy engine created successfully.
2025-07-04 11:02:06.233 | INFO     | src.agent.adapters.database:connect:90 - Connected to postgres database
2025-07-04 11:02:06.485 | INFO     | src.agent.adapters.database:disconnect:101 - Disconnected from database
Authentication error: Langfuse client initialized without public_key. Client will be disabled. Provide a public_key parameter or set LANGFUSE_PUBLIC_KEY environment variable. 
Authentication error: Langfuse client initialized without public_key. Client will be disabled. Provide a public_key parameter or set LANGFUSE_PUBLIC_KEY environment variable. 
Context error: No active span in current context. Operations that depend on an active span will be skipped. Ensure spans are created with start_as_current_span() or that you're operating within an active span context.
Authentication error: Langfuse client initialized without public_key. Client will be d

In [70]:
print(pairs[i][0])
print("-" * 100)

print(result.sql_query)
print("-" * 100)
print(pairs[i][1])

What are the top 5 customers by order value im ersten halbjahr 2024, including their order count?
----------------------------------------------------------------------------------------------------
SELECT
    c.customer_id,
    c.company_name,
    COUNT(DISTINCT o.order_id) AS order_count,
    SUM(o.grand_total_amount) AS total_order_value,
    AVG(o.grand_total_amount) AS average_order_value
FROM customers c
JOIN orders o ON c.customer_id = o.customer_id
WHERE o.order_date BETWEEN '2024-01-01' AND '2024-06-30'
    AND o.status NOT IN ('CANCELLED', 'PENDING')
GROUP BY c.customer_id, c.company_name
ORDER BY total_order_value DESC
LIMIT 5;
----------------------------------------------------------------------------------------------------
SELECT
    c.customer_id,
    c.company_name,
    COUNT(DISTINCT o.order_id) AS order_count,
    SUM(o.grand_total_amount) AS total_order_value,
    o.currency_code,
    AVG(o.grand_total_amount) AS average_order_value
FROM customers c
JOIN orders o ON

### Hard

In [72]:
pairs = [
    (
        "Find the top 5 products (by total quantity sold) in the last 90 days, including their total revenue and category.",
        """SELECT
    p.name AS product_name,
    pc.name AS category_name,
    SUM(od.quantity) AS total_quantity_sold,
    SUM(od.final_unit_price * od.quantity) AS total_revenue
FROM order_details od
JOIN orders o ON od.order_id = o.order_id
JOIN products p ON od.product_id = p.product_id
JOIN product_categories pc ON p.category_id = pc.category_id
WHERE o.order_date >= CURRENT_DATE - INTERVAL '90 days'
GROUP BY p.product_id, p.name, pc.name
ORDER BY total_quantity_sold DESC
LIMIT 5;
""",
    ),
    (
        "For each active supplier, list the total number of products they supply, total units ordered (across all POs), and total cost spent (in their cost currency).",
        """WITH supplier_product_count AS (
    -- Step 1: Correctly count all *currently suppliable* products for each supplier.
    SELECT
        supplier_id,
        COUNT(product_id) AS total_products_supplied
    FROM product_suppliers
    WHERE (end_date IS NULL OR end_date >= CURRENT_DATE)
    GROUP BY supplier_id
),
supplier_spending AS (
    -- Step 2: Correctly calculate total spending from historical purchase orders.
    SELECT
        po.supplier_id,
        po.currency_code,
        SUM(pod.quantity) AS total_units_ordered,
        SUM(pod.quantity * pod.unit_cost) AS total_spent
    FROM purchase_orders po
    JOIN purchase_order_details pod ON po.po_id = pod.po_id
    GROUP BY po.supplier_id, po.currency_code
)
-- Step 3: Join everything together, starting with ALL active suppliers.
SELECT
    s.company_name,
    COALESCE(spc.total_products_supplied, 0) AS total_products_supplied,
    COALESCE(ss.total_units_ordered, 0) AS total_units_ordered,
    COALESCE(ss.total_spent, 0) AS total_spent,
    ss.currency_code
FROM suppliers s
LEFT JOIN supplier_product_count spc ON s.supplier_id = spc.supplier_id
LEFT JOIN supplier_spending ss ON s.supplier_id = ss.supplier_id
WHERE s.is_active = TRUE
ORDER BY s.company_name, ss.currency_code;
""",
    ),
    (
        "List all employees, their salary (in local currency), and its equivalent in USD using the most recent exchange rate.",
        """SELECT
    e.name AS employee_name,
    e.salary,
    e.salary_currency_code,
    er.rate AS to_usd_rate,
    ROUND(e.salary * er.rate, 2) AS salary_in_usd
FROM employees e
JOIN (
    SELECT from_currency, rate
    FROM exchange_rates
    WHERE to_currency = 'USD'
      AND (from_currency, effective_date) IN (
          SELECT from_currency, MAX(effective_date)
          FROM exchange_rates
          WHERE to_currency = 'USD'
          GROUP BY from_currency
      )
) er ON e.salary_currency_code = er.from_currency
WHERE e.salary IS NOT NULL;
""",
    ),
    (
        """Generate a report on customer purchasing behavior. For each customer who has made more than one order, calculate their total lifetime value (total spending), the number of orders they've placed, the date of their first and most recent order, and the average number of days between their consecutive orders""",
        """WITH customer_order_timing AS (
    -- Step 1: For each order, find the date of the *previous* order from the same customer.
    SELECT
        customer_id,
        order_id,
        order_date,
        grand_total_amount,
        -- The LAG window function gets a value from a previous row in the partition.
        -- Here, it gets the previous order_date for this customer.
        LAG(order_date, 1) OVER (PARTITION BY customer_id ORDER BY order_date) AS previous_order_date
    FROM
        orders
)
-- Step 2: Aggregate the results for each customer to calculate the final metrics.
SELECT
    c.company_name,
    COUNT(cot.order_id) AS total_orders,
    SUM(cot.grand_total_amount) AS lifetime_value,
    MIN(cot.order_date) AS first_order_date,
    MAX(cot.order_date) AS last_order_date,
    -- Calculate the average days between orders.
    -- In PostgreSQL, subtracting two dates gives an integer number of days directly.
    ROUND(AVG(cot.order_date - cot.previous_order_date), 2) AS avg_days_between_orders
FROM
    customer_order_timing cot
JOIN
    customers c ON cot.customer_id = c.customer_id
WHERE
    cot.previous_order_date IS NOT NULL -- This filters for customers with at least two orders.
GROUP BY
    c.customer_id, c.company_name
ORDER BY
    lifetime_value DESC;""",
    ),
    (
        """Find the total number of unique products under the 'Electronics' category and all of its sub-categories combined""",
        """WITH RECURSIVE category_hierarchy AS (
    -- Anchor member: Select the starting category
    SELECT
        category_id,
        name,
        parent_category
    FROM
        product_categories
    WHERE
        name = 'Electronics' -- Or any top-level category you want to query

    UNION ALL

    -- Recursive member: Find all child categories
    SELECT
        pc.category_id,
        pc.name,
        pc.parent_category
    FROM
        product_categories pc
    JOIN
        category_hierarchy ch ON pc.parent_category = ch.category_id
)
SELECT
    COUNT(DISTINCT p.product_id) AS total_products_in_hierarchy
FROM
    products p
JOIN
    category_hierarchy ch ON p.category_id = ch.category_id;""",
    ),
    (
        """For each country, identify the top 3 best-selling products based on total revenue in the year 2023. Display the country, product name, their rank, and total revenue in that country""",
        """WITH product_revenue_by_country AS (
    -- Step 1: Calculate total revenue for each product within each country
    SELECT
        co.name AS country_name,
        p.name AS product_name,
        SUM(od.final_unit_price * od.quantity) AS total_revenue
    FROM
        orders o
    JOIN order_details od ON o.order_id = od.order_id
    JOIN products p ON od.product_id = p.product_id
    JOIN addresses sa ON o.shipping_address_id = sa.address_id
    JOIN countries co ON sa.country_id = co.country_id
    WHERE
        EXTRACT(YEAR FROM o.order_date) = 2023
    GROUP BY
        co.name, p.name
),
ranked_product_revenue AS (
    -- Step 2: Rank the products within each country based on revenue
    SELECT
        country_name,
        product_name,
        total_revenue,
        RANK() OVER (PARTITION BY country_name ORDER BY total_revenue DESC) AS revenue_rank
    FROM
        product_revenue_by_country
)
-- Step 3: Filter for the top 3 ranked products in each country
SELECT
    country_name,
    product_name,
    total_revenue,
    revenue_rank
FROM
    ranked_product_revenue
WHERE
    revenue_rank <= 3
ORDER BY
    country_name, revenue_rank;""",
    ),
    (
        """What is the profit margin for each product, comparing the latest selling price against all associated costs (supplier cost, additional costs, and applicable taxes)?""",
        """WITH latest_prices AS (
    SELECT DISTINCT ON (product_id, currency_code)
        product_id,
        currency_code,
        price,
        effective_date
    FROM product_prices
    WHERE end_date IS NULL OR end_date > CURRENT_DATE
    ORDER BY product_id, currency_code, effective_date DESC
),
latest_supplier_costs AS (
    SELECT DISTINCT ON (product_id)
        product_id,
        unit_cost,
        cost_currency_code
    FROM product_suppliers
    WHERE is_preferred = TRUE
        AND (end_date IS NULL OR end_date > CURRENT_DATE)
    ORDER BY product_id, effective_date DESC
),
additional_costs AS (
    SELECT
        product_id,
        SUM(amount) AS total_additional_costs,
        currency_code
    FROM product_costs
    WHERE end_date IS NULL OR end_date > CURRENT_DATE
    GROUP BY product_id, currency_code
),
tax_rates_by_country AS (
    SELECT
        c.country_id,
        c.name AS country_name,
        tt.name AS tax_type,
        tr.rate
    FROM tax_rates tr
    JOIN countries c ON tr.country_id = c.country_id
    JOIN tax_types tt ON tr.tax_type_id = tt.tax_type_id
    WHERE tr.end_date IS NULL OR tr.end_date > CURRENT_DATE
)
SELECT
    p.product_id,
    p.name AS product_name,
    p.sku,
    lp.price AS selling_price,
    lp.currency_code AS selling_currency,
    COALESCE(lsc.unit_cost, 0) AS supplier_cost,
    COALESCE(ac.total_additional_costs, 0) AS additional_costs,
    COALESCE(lsc.unit_cost, 0) + COALESCE(ac.total_additional_costs, 0) AS total_cost,
    lp.price - (COALESCE(lsc.unit_cost, 0) + COALESCE(ac.total_additional_costs, 0)) AS gross_profit,
    CASE
        WHEN lp.price > 0 THEN
            ((lp.price - (COALESCE(lsc.unit_cost, 0) + COALESCE(ac.total_additional_costs, 0))) / lp.price * 100)
        ELSE 0
    END AS profit_margin_percentage
FROM products p
LEFT JOIN latest_prices lp ON p.product_id = lp.product_id
LEFT JOIN latest_supplier_costs lsc ON p.product_id = lsc.product_id
    AND lp.currency_code = lsc.cost_currency_code
LEFT JOIN additional_costs ac ON p.product_id = ac.product_id
    AND lp.currency_code = ac.currency_code
WHERE p.is_active = TRUE
ORDER BY profit_margin_percentage DESC;""",
    ),
    (
        """Analyze order fulfillment performance: Find products with the highest delay between order and delivery, including supplier reliability metrics and impact on customer satisfaction (repeat orders)""",
        """WITH order_fulfillment_times AS (
    SELECT
        od.product_id,
        o.order_id,
        o.customer_id,
        o.order_date,
        o.requested_delivery_date,
        o.shipped_date,
        CASE
            WHEN o.shipped_date IS NOT NULL THEN
                o.shipped_date - o.order_date
            ELSE
                CURRENT_DATE - o.order_date
        END AS actual_fulfillment_days,
        o.requested_delivery_date - o.order_date AS requested_fulfillment_days,
        CASE
            WHEN o.shipped_date IS NOT NULL AND o.requested_delivery_date IS NOT NULL THEN
                o.shipped_date - o.requested_delivery_date
            ELSE NULL
        END AS delay_days
    FROM orders o
    JOIN order_details od ON o.order_id = od.order_id
    WHERE o.status NOT IN ('CANCELLED', 'PENDING')
),
product_fulfillment_stats AS (
    SELECT
        product_id,
        COUNT(*) AS total_orders,
        AVG(actual_fulfillment_days) AS avg_fulfillment_days,
        AVG(delay_days) AS avg_delay_days,
        COUNT(CASE WHEN delay_days > 0 THEN 1 END) AS delayed_orders,
        COUNT(CASE WHEN delay_days > 0 THEN 1 END)::NUMERIC / COUNT(*)::NUMERIC * 100 AS delay_percentage
    FROM order_fulfillment_times
    GROUP BY product_id
),
supplier_performance AS (
    SELECT
        ps.product_id,
        s.supplier_id,
        s.company_name AS supplier_name,
        ps.lead_time_days AS promised_lead_time,
        COUNT(po.po_id) AS total_pos,
        AVG(
            CASE
                WHEN po.received_date IS NOT NULL THEN
                    po.received_date - po.order_date
                ELSE NULL
            END
        ) AS actual_avg_lead_time,
        COUNT(
            CASE
                WHEN po.received_date IS NOT NULL
                AND (po.received_date - po.order_date) > ps.lead_time_days
                THEN 1
            END
        )::NUMERIC / NULLIF(COUNT(CASE WHEN po.received_date IS NOT NULL THEN 1 END), 0)::NUMERIC * 100 AS late_delivery_percentage
    FROM product_suppliers ps
    JOIN suppliers s ON ps.supplier_id = s.supplier_id
    LEFT JOIN purchase_order_details pod ON ps.product_id = pod.product_id
    LEFT JOIN purchase_orders po ON pod.po_id = po.po_id AND po.supplier_id = s.supplier_id
    WHERE ps.is_preferred = TRUE AND (ps.end_date IS NULL OR ps.end_date > CURRENT_DATE)
    GROUP BY ps.product_id, s.supplier_id, s.company_name, ps.lead_time_days
),
customer_repeat_behavior AS (
    SELECT
        oft.product_id,
        COUNT(DISTINCT oft.customer_id) AS unique_customers,
        COUNT(DISTINCT CASE
            WHEN EXISTS (
                SELECT 1 FROM order_fulfillment_times oft2
                WHERE oft2.customer_id = oft.customer_id
                AND oft2.product_id = oft.product_id
                AND oft2.order_date > oft.order_date
            ) THEN oft.customer_id
        END) AS repeat_customers,
        COUNT(DISTINCT CASE
            WHEN oft.delay_days > 0 AND EXISTS (
                SELECT 1 FROM order_fulfillment_times oft2
                WHERE oft2.customer_id = oft.customer_id
                AND oft2.product_id = oft.product_id
                AND oft2.order_date > oft.order_date
            ) THEN oft.customer_id
        END)::NUMERIC /
        NULLIF(COUNT(DISTINCT CASE WHEN oft.delay_days > 0 THEN oft.customer_id END), 0)::NUMERIC * 100 AS repeat_rate_after_delay
    FROM order_fulfillment_times oft
    GROUP BY oft.product_id
)
SELECT
    p.product_id,
    p.name AS product_name,
    p.sku,
    ROUND(pfs.avg_fulfillment_days::NUMERIC, 1) AS avg_fulfillment_days,
    ROUND(pfs.avg_delay_days::NUMERIC, 1) AS avg_delay_days,
    pfs.total_orders,
    pfs.delayed_orders,
    ROUND(pfs.delay_percentage, 1) AS delay_percentage,
    sp.supplier_name,
    sp.promised_lead_time,
    ROUND(sp.actual_avg_lead_time::NUMERIC, 1) AS supplier_actual_lead_time,
    ROUND(sp.late_delivery_percentage, 1) AS supplier_late_percentage,
    crb.unique_customers,
    crb.repeat_customers,
    ROUND(crb.repeat_rate_after_delay, 1) AS repeat_rate_after_delay_pct,
    CASE
        WHEN pfs.delay_percentage > 20 AND sp.late_delivery_percentage > 30 THEN 'Critical - Both fulfillment and supplier issues'
        WHEN pfs.delay_percentage > 20 THEN 'High - Fulfillment issues'
        WHEN sp.late_delivery_percentage > 30 THEN 'Medium - Supplier reliability issues'
        ELSE 'Low - Acceptable performance'
    END AS risk_assessment
FROM products p
JOIN product_fulfillment_stats pfs ON p.product_id = pfs.product_id
LEFT JOIN supplier_performance sp ON p.product_id = sp.product_id
LEFT JOIN customer_repeat_behavior crb ON p.product_id = crb.product_id
WHERE pfs.total_orders >= 5  -- Only products with meaningful order volume
ORDER BY pfs.avg_delay_days DESC NULLS LAST, pfs.delay_percentage DESC
LIMIT 20;""",
    ),
    (
        """Find all employees who manage teams larger than the company average, showing their team size, total team salary cost (converted to USD), and hierarchical organization structure""",
        """WITH RECURSIVE employee_hierarchy AS (
    -- Base case: top-level employees (no manager)
    SELECT
        employee_id,
        name,
        manager_id,
        role_id,
        salary,
        salary_currency_code,
        1 AS level,
        employee_id::TEXT AS path,
        name::TEXT AS name_path
    FROM employees
    WHERE manager_id IS NULL AND is_active = TRUE

    UNION ALL

    -- Recursive case
    SELECT
        e.employee_id,
        e.name,
        e.manager_id,
        e.role_id,
        e.salary,
        e.salary_currency_code,
        eh.level + 1,
        eh.path || '->' || e.employee_id::TEXT,
        eh.name_path || '->' || e.name
    FROM employees e
    JOIN employee_hierarchy eh ON e.manager_id = eh.employee_id
    WHERE e.is_active = TRUE
),
team_sizes AS (
    SELECT
        manager_id,
        COUNT(*) AS direct_reports
    FROM employees
    WHERE manager_id IS NOT NULL AND is_active = TRUE
    GROUP BY manager_id
),
latest_exchange_rates AS (
    SELECT DISTINCT ON (from_currency, to_currency)
        from_currency,
        to_currency,
        rate
    FROM exchange_rates
    WHERE to_currency = 'USD'
    ORDER BY from_currency, to_currency, effective_date DESC
),
team_salary_costs AS (
    SELECT
        m.employee_id AS manager_id,
        m.name AS manager_name,
        COUNT(e.employee_id) AS team_size,
        SUM(
            CASE
                WHEN e.salary_currency_code = 'USD' THEN e.salary
                ELSE e.salary * COALESCE(er.rate, 1)
            END
        ) AS total_team_salary_usd
    FROM employees m
    JOIN employees e ON m.employee_id = e.manager_id
    LEFT JOIN latest_exchange_rates er ON e.salary_currency_code = er.from_currency
    WHERE m.is_active = TRUE AND e.is_active = TRUE
    GROUP BY m.employee_id, m.name
),
company_stats AS (
    SELECT
        AVG(direct_reports::NUMERIC) AS avg_team_size
    FROM team_sizes
)
SELECT
    tsc.manager_id,
    tsc.manager_name,
    r.name AS manager_role,
    tsc.team_size,
    ROUND(cs.avg_team_size, 2) AS company_avg_team_size,
    tsc.total_team_salary_usd,
    eh.level AS org_level,
    eh.name_path AS reporting_chain
FROM team_salary_costs tsc
JOIN employees emp ON tsc.manager_id = emp.employee_id
JOIN roles r ON emp.role_id = r.role_id
JOIN employee_hierarchy eh ON tsc.manager_id = eh.employee_id
CROSS JOIN company_stats cs
WHERE tsc.team_size > cs.avg_team_size
ORDER BY tsc.team_size DESC, eh.level;""",
    ),
]

In [77]:
i = 1
result = controller.process_question(pairs[i])

2025-07-04 11:07:05.069 | INFO     | src.agent.adapters.database:_get_connection:77 - SQLAlchemy engine created successfully.
2025-07-04 11:07:05.070 | INFO     | src.agent.adapters.database:connect:90 - Connected to postgres database
2025-07-04 11:07:05.138 | INFO     | src.agent.adapters.database:disconnect:101 - Disconnected from database
Authentication error: Langfuse client initialized without public_key. Client will be disabled. Provide a public_key parameter or set LANGFUSE_PUBLIC_KEY environment variable. 
Authentication error: Langfuse client initialized without public_key. Client will be disabled. Provide a public_key parameter or set LANGFUSE_PUBLIC_KEY environment variable. 
Context error: No active span in current context. Operations that depend on an active span will be skipped. Ensure spans are created with start_as_current_span() or that you're operating within an active span context.
Authentication error: Langfuse client initialized without public_key. Client will be d

In [78]:
print(pairs[i][0])
print("-" * 100)

print(result.sql_query)
print("-" * 100)
print(pairs[i][1])

For each active supplier, list the total number of products they supply, total units ordered (across all POs), and total cost spent (in their cost currency).
----------------------------------------------------------------------------------------------------
SELECT
    s.company_name,
    COUNT(DISTINCT ps.product_id) AS total_products_supplied,
    COALESCE(SUM(pod.quantity), 0) AS total_units_ordered,
    COALESCE(SUM(pod.quantity * ps.unit_cost), 0) AS total_spent,
    ps.cost_currency_code
FROM suppliers s
JOIN product_suppliers ps ON s.supplier_id = ps.supplier_id
LEFT JOIN purchase_order_details pod ON ps.product_id = pod.product_id
LEFT JOIN purchase_orders po ON pod.po_id = po.po_id AND po.supplier_id = s.supplier_id
WHERE s.is_active = TRUE
  AND (ps.end_date IS NULL OR ps.end_date >= CURRENT_DATE)
  AND (ps.effective_date IS NULL OR ps.effective_date <= CURRENT_DATE)
GROUP BY s.supplier_id, s.company_name, ps.cost_currency_code;
-----------------------------------------------

In [79]:
i = 2
result = controller.process_question(pairs[i])

2025-07-04 11:10:36.266 | INFO     | src.agent.adapters.database:_get_connection:77 - SQLAlchemy engine created successfully.
2025-07-04 11:10:36.266 | INFO     | src.agent.adapters.database:connect:90 - Connected to postgres database
2025-07-04 11:10:36.397 | INFO     | src.agent.adapters.database:disconnect:101 - Disconnected from database
Authentication error: Langfuse client initialized without public_key. Client will be disabled. Provide a public_key parameter or set LANGFUSE_PUBLIC_KEY environment variable. 
Authentication error: Langfuse client initialized without public_key. Client will be disabled. Provide a public_key parameter or set LANGFUSE_PUBLIC_KEY environment variable. 
Context error: No active span in current context. Operations that depend on an active span will be skipped. Ensure spans are created with start_as_current_span() or that you're operating within an active span context.
Authentication error: Langfuse client initialized without public_key. Client will be d

In [80]:
print(pairs[i][0])
print("-" * 100)

print(result.sql_query)
print("-" * 100)
print(pairs[i][1])

List all employees, their salary (in local currency), and its equivalent in USD using the most recent exchange rate.
----------------------------------------------------------------------------------------------------
SELECT
    e.name AS employee_name,
    e.salary,
    e.salary_currency_code,
    er.rate AS to_usd_rate,
    ROUND(e.salary * er.rate, 2) AS salary_in_usd
FROM employees e
JOIN (
    SELECT from_currency, rate
    FROM exchange_rates
    WHERE to_currency = 'USD'
      AND (from_currency, effective_date) IN (
          SELECT from_currency, MAX(effective_date)
          FROM exchange_rates
          WHERE to_currency = 'USD'
          GROUP BY from_currency
      )
) er ON e.salary_currency_code = er.from_currency
WHERE e.salary IS NOT NULL;
----------------------------------------------------------------------------------------------------
SELECT
    e.name AS employee_name,
    e.salary,
    e.salary_currency_code,
    er.rate AS to_usd_rate,
    ROUND(e.salary * er.ra

In [81]:
i = 4
result = controller.process_question(pairs[i])

2025-07-04 11:13:58.209 | INFO     | src.agent.adapters.database:_get_connection:77 - SQLAlchemy engine created successfully.
2025-07-04 11:13:58.209 | INFO     | src.agent.adapters.database:connect:90 - Connected to postgres database
2025-07-04 11:13:58.343 | INFO     | src.agent.adapters.database:disconnect:101 - Disconnected from database
Authentication error: Langfuse client initialized without public_key. Client will be disabled. Provide a public_key parameter or set LANGFUSE_PUBLIC_KEY environment variable. 
Authentication error: Langfuse client initialized without public_key. Client will be disabled. Provide a public_key parameter or set LANGFUSE_PUBLIC_KEY environment variable. 
Context error: No active span in current context. Operations that depend on an active span will be skipped. Ensure spans are created with start_as_current_span() or that you're operating within an active span context.
Authentication error: Langfuse client initialized without public_key. Client will be d

In [82]:
print(pairs[i][0])
print("-" * 100)

print(result.sql_query)
print("-" * 100)
print(pairs[i][1])

Find the total number of unique products under the 'Electronics' category and all of its sub-categories combined
----------------------------------------------------------------------------------------------------
WITH RECURSIVE category_hierarchy AS (
    -- Anchor member: Select the starting category
    SELECT
        category_id,
        name,
        parent_category
    FROM
        product_categories
    WHERE
        name = 'Electronics'

    UNION ALL

    -- Recursive member: Find all child categories
    SELECT
        pc.category_id,
        pc.name,
        pc.parent_category
    FROM
        product_categories pc
    JOIN
        category_hierarchy ch ON pc.parent_category = ch.category_id
)
SELECT
    COUNT(DISTINCT p.product_id) AS total_products_in_hierarchy
FROM
    products p
JOIN
    category_hierarchy ch ON p.category_id = ch.category_id;
----------------------------------------------------------------------------------------------------
WITH RECURSIVE category_hier

In [86]:
i = 6
result = controller.process_question(pairs[i])

2025-07-04 11:20:06.021 | INFO     | src.agent.adapters.database:_get_connection:77 - SQLAlchemy engine created successfully.
2025-07-04 11:20:06.022 | INFO     | src.agent.adapters.database:connect:90 - Connected to postgres database
2025-07-04 11:20:06.099 | INFO     | src.agent.adapters.database:disconnect:101 - Disconnected from database
Authentication error: Langfuse client initialized without public_key. Client will be disabled. Provide a public_key parameter or set LANGFUSE_PUBLIC_KEY environment variable. 
Authentication error: Langfuse client initialized without public_key. Client will be disabled. Provide a public_key parameter or set LANGFUSE_PUBLIC_KEY environment variable. 
Context error: No active span in current context. Operations that depend on an active span will be skipped. Ensure spans are created with start_as_current_span() or that you're operating within an active span context.
Authentication error: Langfuse client initialized without public_key. Client will be d

In [87]:
print(pairs[i][0])
print("-" * 100)

print(result.sql_query)
print("-" * 100)
print(pairs[i][1])

What is the profit margin for each product, comparing the latest selling price against all associated costs (supplier cost, additional costs, and applicable taxes)?
----------------------------------------------------------------------------------------------------
WITH latest_prices AS (
    SELECT DISTINCT ON (product_id, currency_code)
        product_id,
        currency_code,
        price,
        effective_date
    FROM product_prices
    WHERE end_date IS NULL OR end_date > CURRENT_DATE
    ORDER BY product_id, currency_code, effective_date DESC
),
latest_supplier_costs AS (
    SELECT DISTINCT ON (product_id)
        product_id,
        unit_cost,
        cost_currency_code
    FROM product_suppliers
    WHERE is_preferred = TRUE
        AND (end_date IS NULL OR end_date > CURRENT_DATE)
    ORDER BY product_id, effective_date DESC
),
additional_costs AS (
    SELECT
        product_id,
        SUM(amount) AS total_additional_costs,
        currency_code
    FROM product_costs


In [88]:
i = 8
result = controller.process_question(pairs[i])

2025-07-04 11:22:33.957 | INFO     | src.agent.adapters.database:_get_connection:77 - SQLAlchemy engine created successfully.
2025-07-04 11:22:33.958 | INFO     | src.agent.adapters.database:connect:90 - Connected to postgres database
2025-07-04 11:22:34.094 | INFO     | src.agent.adapters.database:disconnect:101 - Disconnected from database
Authentication error: Langfuse client initialized without public_key. Client will be disabled. Provide a public_key parameter or set LANGFUSE_PUBLIC_KEY environment variable. 
Authentication error: Langfuse client initialized without public_key. Client will be disabled. Provide a public_key parameter or set LANGFUSE_PUBLIC_KEY environment variable. 
Context error: No active span in current context. Operations that depend on an active span will be skipped. Ensure spans are created with start_as_current_span() or that you're operating within an active span context.
Authentication error: Langfuse client initialized without public_key. Client will be d

In [89]:
print(pairs[i][0])
print("-" * 100)

print(result.sql_query)
print("-" * 100)
print(pairs[i][1])

Find all employees who manage teams larger than the company average, showing their team size, total team salary cost (converted to USD), and hierarchical organization structure
----------------------------------------------------------------------------------------------------
WITH RECURSIVE employee_hierarchy AS (
    -- Base case: top-level employees (no manager)
    SELECT
        employee_id,
        name,
        manager_id,
        role_id,
        salary,
        salary_currency_code,
        1 AS level,
        employee_id::TEXT AS path,
        name::TEXT AS name_path
    FROM employees
    WHERE manager_id IS NULL AND is_active = TRUE

    UNION ALL

    -- Recursive case
    SELECT
        e.employee_id,
        e.name,
        e.manager_id,
        e.role_id,
        e.salary,
        e.salary_currency_code,
        eh.level + 1,
        eh.path || '->' || e.employee_id::TEXT,
        eh.name_path || '->' || e.name
    FROM employees e
    JOIN employee_hierarchy eh ON e.ma